In [0]:
# Load data
events = spark.read.csv("/path/to/sample.csv", header=True, inferSchema=True)

# Basic operations
events.select("event_type", "product_name", "price").show(10)
events.filter("price > 100").count()
events.groupBy("event_type").count().show()
top_brands = events.groupBy("brand").count().orderBy("count", ascending=False).limit(5)

In [0]:
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)
# Basic operations
events.select("event_type","category_id","category_code","price").show(10)
events.filter("price > 100").count()
events.groupBy("event_type").count().show()
events.orderBy("price", ascending=False).show(10)
top_brands = events.groupBy("brand").count().orderBy("count", ascending=False).limit(5)

In [0]:
%python

events.write.mode("overwrite").option("header", "true").csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct-export.csv")

In [0]:
%python
events.coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct-export")

In [0]:
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)
# Basic operations
events.select("event_type","category_id","category_code","price").show(10)
events.filter("price > 100").count()
events.groupBy("event_type").count().show()
events.orderBy("price", ascending=False).show(10)
top_brands = events.groupBy("brand").count().orderBy("count", ascending=False).limit(5)

events.coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov-export")